# Ajustes dos dados com Random Forest

## Importando as bibliotecas

In [193]:
import pandas as pd
import numpy as np
import ast
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

## Criando a engine de conexão com o banco de dados

In [194]:
path_db = '../db/filmes.db'

engine = create_engine(f'sqlite:///{path_db}')

# Carregando o DataFrame

Vamos carregar o DataFrame partindo da databela `filmes` no banco de dados. 

In [195]:
df = pd.read_sql('SELECT * from filmes', con=engine)

# Criando novas colunas

Partindo de que as colunas `genres` e `release_date` do `df` não servirão como `features` para inputar a coluna `budget`, criaremos as colunas numéricas `num_genres` e `release_year`. Para fazermos isso, primeiramente iremos transformar em `lista` os elementos da coluna `genres` que até então são `string de lista`. 

In [196]:
df['genres'] = df['genres'].apply(ast.literal_eval)

Criando a coluna `num_genres`

In [197]:
df['num_genres'] = df['genres'].apply(lambda x: len(x) if isinstance(x, list) else 0)

Convertendo a coluna `release_date` para `datetimes`

In [198]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

Criando a coluna `release_year`

In [199]:
df['release_year'] = df['release_date'].dt.year

In [200]:
df

,id,title,original_title,original_language,overview,budget,revenue,runtime,release_date,genres,popularity,vote_average,vote_count,num_genres,release_year
0,411405,Small Crimes,Small Crimes,en,"A disgraced former cop, fresh off a six-year p...",0.0,0.0,95.0,2017-04-28,"[Drama, Comedy, Thriller, Crime]",7.219022,5.8,55.0,4,2017.0
1,42492,Up the Sandbox,Up the Sandbox,en,"A young wife and mother, bored with day-to-day...",0.0,0.0,97.0,1972-12-21,"[Drama, Comedy]",0.138450,7.3,2.0,2,1972.0
2,12143,Bad Lieutenant,Bad Lieutenant,en,"While investigating a young nun's rape, a corr...",1000000.0,2019469.0,96.0,1992-09-16,"[Crime, Drama]",6.417037,6.9,162.0,2,1992.0
3,9976,Satan's Little Helper,Satan's Little Helper,en,A naïve young boy unknowingly becomes the pawn...,0.0,0.0,100.0,2004-01-01,"[Horror, Romance, Comedy]",2.233189,5.0,42.0,3,2004.0
4,46761,Sitcom,Sitcom,fr,The adventures of an upper-class suburban fami...,0.0,0.0,80.0,1998-05-27,"[Comedy, Drama, Thriller]",1.800582,6.4,27.0,3,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3093,Basic Instinct 2,Basic Instinct 2,en,Novelist Catherine Tramell is once again in tr...,70000000.0,38629478.0,114.0,2006-03-29,"[Crime, Mystery, Thriller]",15.321902,4.6,183.0,3,2006.0
4996,246127,Every Thing Will Be Fine,Every Thing Will Be Fine,en,"One day, driving aimlessly around the outskirt...",0.0,8034.0,118.0,2015-04-02,[Drama],5.723103,5.3,79.0,1,2015.0
4997,9803,Seven Dwarfs,7 Zwerge - Männer allein im Wald,de,The Seven Dwarves live deep within a female-fr...,0.0,0.0,95.0,2004-10-28,[Comedy],4.582736,5.2,70.0,1,2004.0
4998,336970,True Siblings,Syskonsalt,sv,"The siblings Linus, 19-years-old, who are taki...",0.0,0.0,58.0,2000-09-13,"[Drama, TV Movie]",2.364355,8.0,2.0,2,2000.0


Agora faremos one hot encoder na coluna `original language` para também inserirmos no treinamento do modelo de florestas aleatórias.

In [201]:
df_encoder = pd.get_dummies(df, columns=['original_language'])

In [202]:
df_encoder

,id,title,original_title,overview,budget,revenue,runtime,release_date,genres,popularity,...,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_uz,original_language_vi,original_language_xx,original_language_zh
0,411405,Small Crimes,Small Crimes,"A disgraced former cop, fresh off a six-year p...",0.0,0.0,95.0,2017-04-28,"[Drama, Comedy, Thriller, Crime]",7.219022,...,False,False,False,False,False,False,False,False,False,False
1,42492,Up the Sandbox,Up the Sandbox,"A young wife and mother, bored with day-to-day...",0.0,0.0,97.0,1972-12-21,"[Drama, Comedy]",0.138450,...,False,False,False,False,False,False,False,False,False,False
2,12143,Bad Lieutenant,Bad Lieutenant,"While investigating a young nun's rape, a corr...",1000000.0,2019469.0,96.0,1992-09-16,"[Crime, Drama]",6.417037,...,False,False,False,False,False,False,False,False,False,False
3,9976,Satan's Little Helper,Satan's Little Helper,A naïve young boy unknowingly becomes the pawn...,0.0,0.0,100.0,2004-01-01,"[Horror, Romance, Comedy]",2.233189,...,False,False,False,False,False,False,False,False,False,False
4,46761,Sitcom,Sitcom,The adventures of an upper-class suburban fami...,0.0,0.0,80.0,1998-05-27,"[Comedy, Drama, Thriller]",1.800582,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3093,Basic Instinct 2,Basic Instinct 2,Novelist Catherine Tramell is once again in tr...,70000000.0,38629478.0,114.0,2006-03-29,"[Crime, Mystery, Thriller]",15.321902,...,False,False,False,False,False,False,False,False,False,False
4996,246127,Every Thing Will Be Fine,Every Thing Will Be Fine,"One day, driving aimlessly around the outskirt...",0.0,8034.0,118.0,2015-04-02,[Drama],5.723103,...,False,False,False,False,False,False,False,False,False,False
4997,9803,Seven Dwarfs,7 Zwerge - Männer allein im Wald,The Seven Dwarves live deep within a female-fr...,0.0,0.0,95.0,2004-10-28,[Comedy],4.582736,...,False,False,False,False,False,False,False,False,False,False
4998,336970,True Siblings,Syskonsalt,"The siblings Linus, 19-years-old, who are taki...",0.0,0.0,58.0,2000-09-13,"[Drama, TV Movie]",2.364355,...,False,False,False,False,False,False,False,False,False,False


Vamos analizar se temos valores `budget` menores do que zero e a descrição da coluna para entendermos seus valores.

In [203]:
df['budget'][df['budget'] < 0].sum()

0.0

Vemos, portanto, que 50% (__mediana__) tem orçamento abaixo de __8 milhões__, 75% abaixo de __25 milhões__ o que é típico e esperado para um dataset de filmes. Porém, percebemos que há `outliers` muito grandes como o máximo sendo de __100 bilhões__ o que acarreta uma média de __131 milhôes__, bem acima da mediana por conta desses outliers. Além disso, o desvio padrão está em __3.33 bilhões__ o que é altíssimo e confere uma grande variabilidade nos dados devido a outliers.   

In [204]:
df['budget'][df['budget'] > 0].describe()

count    9.000000e+02
mean     1.316451e+08
std      3.332803e+09
min      1.000000e+00
25%      2.000000e+06
50%      8.000000e+06
75%      2.500000e+07
max      1.000000e+11
Name: budget, dtype: float64

# Inputando as colunas Budget e Revenue

In [205]:
df_encoder.columns

Index(['id', 'title', 'original_title', 'overview', 'budget', 'revenue',
       'runtime', 'release_date', 'genres', 'popularity', 'vote_average',
       'vote_count', 'num_genres', 'release_year', 'original_language_ar',
       'original_language_ay', 'original_language_bg', 'original_language_bn',
       'original_language_ca', 'original_language_cn', 'original_language_cs',
       'original_language_da', 'original_language_de', 'original_language_el',
       'original_language_en', 'original_language_es', 'original_language_et',
       'original_language_fa', 'original_language_fi', 'original_language_fr',
       'original_language_he', 'original_language_hi', 'original_language_hr',
       'original_language_hu', 'original_language_id', 'original_language_is',
       'original_language_it', 'original_language_iu', 'original_language_ja',
       'original_language_ka', 'original_language_ko', 'original_language_lv',
       'original_language_mk', 'original_language_ml', 'original_la

Aqui temos as colunas de `features` que servirão para o treinamento e previsão do modelo.

In [206]:
features = ['runtime', 'popularity', 'vote_average', 'vote_count', 'num_genres', 'release_year', 'original_language_ar',
       'original_language_ay', 'original_language_bg', 'original_language_bn',
       'original_language_ca', 'original_language_cn', 'original_language_cs',
       'original_language_da', 'original_language_de', 'original_language_el',
       'original_language_en', 'original_language_es', 'original_language_et',
       'original_language_fa', 'original_language_fi', 'original_language_fr',
       'original_language_he', 'original_language_hi', 'original_language_hr',
       'original_language_hu', 'original_language_id', 'original_language_is',
       'original_language_it', 'original_language_iu', 'original_language_ja',
       'original_language_ka', 'original_language_ko', 'original_language_lv',
       'original_language_mk', 'original_language_ml', 'original_language_mn',
       'original_language_mr', 'original_language_ms', 'original_language_nb',
       'original_language_nl', 'original_language_no', 'original_language_pa',
       'original_language_pl', 'original_language_ps', 'original_language_pt',
       'original_language_ro', 'original_language_ru', 'original_language_sk',
       'original_language_sl', 'original_language_sq', 'original_language_sr',
       'original_language_sv', 'original_language_ta', 'original_language_te',
       'original_language_th', 'original_language_tl', 'original_language_tr',
       'original_language_uk', 'original_language_ur', 'original_language_uz',
       'original_language_vi', 'original_language_xx', 'original_language_zh']

A fim de testar se o modelo vai ter bom desempenho na inputação de valores em `budget` e tomando por base a análise feita anteriormente nesse coluna, antes de inputar vamos avaliar o desempenho do modelo de Florestas Aleatórias pelo Erro Absoluto Médio `(MAE)`. Essa métrica é uma das mais indicadas para inputar valores em colunas por regressão de Florestas Aleatórias, pois são robustos a outliers já que tratam todos os erros da mesma forma controlando o erro médio. 

In [207]:
def avaliar_modelo(df, coluna_alvo):
    
    df_completo = df[df[coluna_alvo] != 0]
    
    X = df_completo.drop(columns=[coluna_alvo])
    y = df_completo[coluna_alvo]
    
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    maes = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
    
        y_pred = model.predict(X_test)
    
        mae = mean_absolute_error(y_pred, y_test)
        maes.append(mae)
    
    return {'mae_medio': np.mean(maes),
            'mae_std': np.std(maes),
            'maes': maes}

Temos, portanto, que para a coluna `budget` o modelo erra aproximadamente `R$ 178397270.37` em média (__MAE médio__) ao prever o valor na inputação, muito maior do que um valor típico de orçamento de um filme que geralmente está entre __3 milhões__ e __50 milhões__ o que é inaceitável para qualquer aplicação prática de inputação. Muito provavelmente isso ocorreu por conta dos outliers no conjunto de teste para os quais o modelo não conseguiu prever corretamente. Logo, o modelo falha gravemente com outliers mesmo que tenha desempenho razoável na maioria dos casos.  

In [208]:
mae_budget = avaliar_modelo(df_encoder[features + ['budget']], 'budget')
mae_budget

{'mae_medio': 178397270.3682778,
 'mae_std': 329235369.01105577,
 'maes': [7687096.507111111,
  14396465.060999997,
  1125155120.639111,
  54793709.08088888,
  266562233.38711113,
  11345948.283333333,
  246107375.52311116,
  11241932.837777779,
  24641028.29644445,
  22041794.06688889]}

Para tratarmos da distribuição assimétrica dos dados vamos então aprimorar nossa função de avaliar o modelo aplicando uma transformação logarítimica. Isso reduz o impacto dos valores extremos pois o modelo passa a aprender proporções e não valores absolutos.

In [209]:
def avaliar_modelo_log(df, coluna_alvo):
    
    df_completo = df[df[coluna_alvo] != 0]
    
    X = df_completo.drop(columns=[coluna_alvo])
    y = df_completo[coluna_alvo]
    
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    log_maes = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
    
        y_pred = model.predict(X_test)
        
        log_y_test = np.log1p(y_test)
        log_y_pred = np.log1p(y_pred)
    
        log_mae = mean_absolute_error(log_y_pred, log_y_test)
        log_maes.append(log_mae)
    
    return {'log_mae_medio': np.mean(log_maes),
            'log_mae_std': np.std(log_maes),
            'log_maes': log_maes}

In [210]:
log_mae_budget = avaliar_modelo_log(df_encoder[features + ['budget']], 'budget')
log_mae_budget

{'log_mae_medio': 1.3201484890944148,
 'log_mae_std': 0.21469090087070783,
 'log_maes': [1.1519760045594163,
  1.2887373442218995,
  1.5853842778637575,
  1.480092237154925,
  1.4988956252258216,
  1.49294389436308,
  1.3353976000413192,
  0.8123411731246748,
  1.1996450266210934,
  1.3560717077681603]}

O modelo tem uma melhora significativa pois na escala original o erro logarítmico de `1.32` equivale a errar por um fator de `2.74`, ou seja, o modelo, em média prevê valores `2.74` vezes acima ou abaixo do valor real o que é mais razoavel do que errar por centenas de milhões ou bilhões como anteriormente sem a transformação log. Além disso, o __desvio padrão__ do `log_mae` ser aproximadamente `0.21` nos indica uma consistencia bem maior entre os `folds` e o modelo aprendeu padrões mais coerentes com a realidade dos dados. 

In [211]:
np.expm1(1.32)

2.7434213772608627

Agora feita a análise dos erros com transformação logarítmica, vamos de fato inputar os valores em `budget`.

In [212]:
def inputar_coluna_log(df, coluna_alvo, colunas):
    
    valido = ~df[colunas].isna().any(axis=1)
    
    X_train = df.loc[valido & (df[coluna_alvo] != 0), colunas]
    Y_train = df.loc[valido & (df[coluna_alvo] != 0), coluna_alvo]
    X_pred = df.loc[valido & (df[coluna_alvo] == 0), colunas]
    
    Y_train_log = np.log1p(Y_train)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, Y_train_log)
    
    Y_pred_log = model.predict(X_pred)
    
    Y_pred_original = np.expm1(Y_pred_log)
    
    df.loc[X_pred.index, coluna_alvo] = Y_pred_original
    
    return df

In [213]:
df_budget_inputado = inputar_coluna_log(df_encoder, 'budget', features)
df_budget_inputado

,id,title,original_title,overview,budget,revenue,runtime,release_date,genres,popularity,...,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_uz,original_language_vi,original_language_xx,original_language_zh
0,411405,Small Crimes,Small Crimes,"A disgraced former cop, fresh off a six-year p...",8.201666e+06,0.0,95.0,2017-04-28,"[Drama, Comedy, Thriller, Crime]",7.219022,...,False,False,False,False,False,False,False,False,False,False
1,42492,Up the Sandbox,Up the Sandbox,"A young wife and mother, bored with day-to-day...",3.543621e+05,0.0,97.0,1972-12-21,"[Drama, Comedy]",0.138450,...,False,False,False,False,False,False,False,False,False,False
2,12143,Bad Lieutenant,Bad Lieutenant,"While investigating a young nun's rape, a corr...",1.000000e+06,2019469.0,96.0,1992-09-16,"[Crime, Drama]",6.417037,...,False,False,False,False,False,False,False,False,False,False
3,9976,Satan's Little Helper,Satan's Little Helper,A naïve young boy unknowingly becomes the pawn...,1.890390e+06,0.0,100.0,2004-01-01,"[Horror, Romance, Comedy]",2.233189,...,False,False,False,False,False,False,False,False,False,False
4,46761,Sitcom,Sitcom,The adventures of an upper-class suburban fami...,8.550376e+05,0.0,80.0,1998-05-27,"[Comedy, Drama, Thriller]",1.800582,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3093,Basic Instinct 2,Basic Instinct 2,Novelist Catherine Tramell is once again in tr...,7.000000e+07,38629478.0,114.0,2006-03-29,"[Crime, Mystery, Thriller]",15.321902,...,False,False,False,False,False,False,False,False,False,False
4996,246127,Every Thing Will Be Fine,Every Thing Will Be Fine,"One day, driving aimlessly around the outskirt...",9.853274e+06,8034.0,118.0,2015-04-02,[Drama],5.723103,...,False,False,False,False,False,False,False,False,False,False
4997,9803,Seven Dwarfs,7 Zwerge - Männer allein im Wald,The Seven Dwarves live deep within a female-fr...,9.674091e+06,0.0,95.0,2004-10-28,[Comedy],4.582736,...,False,False,False,False,False,False,False,False,False,False
4998,336970,True Siblings,Syskonsalt,"The siblings Linus, 19-years-old, who are taki...",1.060106e+06,0.0,58.0,2000-09-13,"[Drama, TV Movie]",2.364355,...,False,False,False,False,False,False,False,False,False,False


In [214]:
(df_budget_inputado['budget'] == 0).sum()

87

# Salvando o novo dataset no banco de dados

Primeiro vamos desfazer o one hot encoder para a coluna de `original_language` de antes.

In [215]:
df['budget'] = df_budget_inputado['budget']

In [216]:
df

,id,title,original_title,original_language,overview,budget,revenue,runtime,release_date,genres,popularity,vote_average,vote_count,num_genres,release_year
0,411405,Small Crimes,Small Crimes,en,"A disgraced former cop, fresh off a six-year p...",8.201666e+06,0.0,95.0,2017-04-28,"[Drama, Comedy, Thriller, Crime]",7.219022,5.8,55.0,4,2017.0
1,42492,Up the Sandbox,Up the Sandbox,en,"A young wife and mother, bored with day-to-day...",3.543621e+05,0.0,97.0,1972-12-21,"[Drama, Comedy]",0.138450,7.3,2.0,2,1972.0
2,12143,Bad Lieutenant,Bad Lieutenant,en,"While investigating a young nun's rape, a corr...",1.000000e+06,2019469.0,96.0,1992-09-16,"[Crime, Drama]",6.417037,6.9,162.0,2,1992.0
3,9976,Satan's Little Helper,Satan's Little Helper,en,A naïve young boy unknowingly becomes the pawn...,1.890390e+06,0.0,100.0,2004-01-01,"[Horror, Romance, Comedy]",2.233189,5.0,42.0,3,2004.0
4,46761,Sitcom,Sitcom,fr,The adventures of an upper-class suburban fami...,8.550376e+05,0.0,80.0,1998-05-27,"[Comedy, Drama, Thriller]",1.800582,6.4,27.0,3,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3093,Basic Instinct 2,Basic Instinct 2,en,Novelist Catherine Tramell is once again in tr...,7.000000e+07,38629478.0,114.0,2006-03-29,"[Crime, Mystery, Thriller]",15.321902,4.6,183.0,3,2006.0
4996,246127,Every Thing Will Be Fine,Every Thing Will Be Fine,en,"One day, driving aimlessly around the outskirt...",9.853274e+06,8034.0,118.0,2015-04-02,[Drama],5.723103,5.3,79.0,1,2015.0
4997,9803,Seven Dwarfs,7 Zwerge - Männer allein im Wald,de,The Seven Dwarves live deep within a female-fr...,9.674091e+06,0.0,95.0,2004-10-28,[Comedy],4.582736,5.2,70.0,1,2004.0
4998,336970,True Siblings,Syskonsalt,sv,"The siblings Linus, 19-years-old, who are taki...",1.060106e+06,0.0,58.0,2000-09-13,"[Drama, TV Movie]",2.364355,8.0,2.0,2,2000.0


Vamos mudar os registros em `genres` para `strings` para salvarmos no banco de dados já que não é permitido salvar como listas. 

In [217]:
df['genres'] = df['genres'].apply(lambda x: str(x))

In [218]:
df

,id,title,original_title,original_language,overview,budget,revenue,runtime,release_date,genres,popularity,vote_average,vote_count,num_genres,release_year
0,411405,Small Crimes,Small Crimes,en,"A disgraced former cop, fresh off a six-year p...",8.201666e+06,0.0,95.0,2017-04-28,"['Drama', 'Comedy', 'Thriller', 'Crime']",7.219022,5.8,55.0,4,2017.0
1,42492,Up the Sandbox,Up the Sandbox,en,"A young wife and mother, bored with day-to-day...",3.543621e+05,0.0,97.0,1972-12-21,"['Drama', 'Comedy']",0.138450,7.3,2.0,2,1972.0
2,12143,Bad Lieutenant,Bad Lieutenant,en,"While investigating a young nun's rape, a corr...",1.000000e+06,2019469.0,96.0,1992-09-16,"['Crime', 'Drama']",6.417037,6.9,162.0,2,1992.0
3,9976,Satan's Little Helper,Satan's Little Helper,en,A naïve young boy unknowingly becomes the pawn...,1.890390e+06,0.0,100.0,2004-01-01,"['Horror', 'Romance', 'Comedy']",2.233189,5.0,42.0,3,2004.0
4,46761,Sitcom,Sitcom,fr,The adventures of an upper-class suburban fami...,8.550376e+05,0.0,80.0,1998-05-27,"['Comedy', 'Drama', 'Thriller']",1.800582,6.4,27.0,3,1998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3093,Basic Instinct 2,Basic Instinct 2,en,Novelist Catherine Tramell is once again in tr...,7.000000e+07,38629478.0,114.0,2006-03-29,"['Crime', 'Mystery', 'Thriller']",15.321902,4.6,183.0,3,2006.0
4996,246127,Every Thing Will Be Fine,Every Thing Will Be Fine,en,"One day, driving aimlessly around the outskirt...",9.853274e+06,8034.0,118.0,2015-04-02,['Drama'],5.723103,5.3,79.0,1,2015.0
4997,9803,Seven Dwarfs,7 Zwerge - Männer allein im Wald,de,The Seven Dwarves live deep within a female-fr...,9.674091e+06,0.0,95.0,2004-10-28,['Comedy'],4.582736,5.2,70.0,1,2004.0
4998,336970,True Siblings,Syskonsalt,sv,"The siblings Linus, 19-years-old, who are taki...",1.060106e+06,0.0,58.0,2000-09-13,"['Drama', 'TV Movie']",2.364355,8.0,2.0,2,2000.0


Agora vamos salvar o novo dataframe no banco de dados.

In [219]:
df.to_sql('filmes', con=engine, if_exists='replace', index=False)

5000